In [1]:
DATA_NAME = 'spaceNet-full' 
TRANSFORM = 'wavelet-horizVert'
CHANNEL = 'green'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_green_horizVert_wavelet_full_spaceNet.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9
2,20586.47,0.12,0.17,-0.05,0.04,0.03,-0.01,-0.00
3,0.12,5586.91,-0.13,-0.02,0.01,-0.00,0.00,-0.00
4,0.17,-0.13,1803.56,-0.01,0.01,0.02,0.00,0.00
5,-0.05,-0.02,-0.01,518.63,-0.01,-0.01,-0.00,-0.00
6,0.04,0.01,0.01,-0.01,123.44,0.00,0.00,0.00
7,0.03,-0.00,0.02,-0.01,0.00,26.48,-0.00,0.00
8,-0.01,0.00,0.00,-0.00,0.00,-0.00,5.57,0.00
9,-0.00,-0.00,0.00,-0.00,0.00,0.00,0.00,1.02


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9
2,1.00000,0.00001,0.00003,-0.00001,0.00003,0.00003,-0.00001,-0.00002
3,0.00001,1.00000,-0.00004,-0.00001,0.00002,-0.00001,0.00002,-0.00006
4,0.00003,-0.00004,1.00000,-0.00001,0.00001,0.00009,0.00000,0.00001
5,-0.00001,-0.00001,-0.00001,1.00000,-0.00003,-0.00006,-0.00001,-0.00001
6,0.00003,0.00002,0.00001,-0.00003,1.00000,0.00002,0.00002,0.00002
7,0.00003,-0.00001,0.00009,-0.00006,0.00002,1.00000,-0.00002,0.00010
8,-0.00001,0.00002,0.00000,-0.00001,0.00002,-0.00002,1.00000,0.00002
9,-0.00002,-0.00006,0.00001,-0.00001,0.00002,0.00010,0.00002,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.3698695086835118

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[5.30697211e+07 3.82944215e+06 3.87421165e+05 3.07423471e+04
 1.63529112e+03 6.71840698e+01 2.26487211e+00 2.44157832e-32]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7
0,0.999047,-0.041674,-0.012424,-0.003549,-0.000842,-0.000180,-0.000038,-0.000007
1,0.040790,0.997222,-0.060280,-0.015379,-0.003622,-0.000778,-0.000163,-0.000031
2,0.014593,0.058583,0.996061,-0.063419,-0.013668,-0.002906,-0.000613,-0.000113
3,0.005020,0.018617,0.061321,0.995807,-0.063765,-0.012760,-0.002673,-0.000491
4,0.001482,0.005469,0.017000,0.061550,0.994985,-0.075272,-0.014900,-0.002729
5,0.000418,0.001546,0.004777,0.016692,0.072160,0.991037,-0.109458,-0.018784
6,0.000129,0.000475,0.001474,0.005131,0.021565,0.102805,0.977371,-0.183533
7,0.000049,0.000180,0.000556,0.001938,0.008135,0.037923,0.180378,0.982830


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0009526588288677473,
 0.0027777806679953887,
 0.003939058024786002,
 0.004192914222411082,
 0.005015034871023616,
 0.0089625832414576,
 0.022629493839448678,
 0.017169781806012585]